<a href="https://colab.research.google.com/github/RobiulArman/-Tea-Landing-Page-With-Tailwind/blob/main/BloomTaxonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers tensorflow pandas sklearn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# Install necessary libraries (if running in Colab or local environment)
# !pip install transformers tensorflow pandas sklearn

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf

# Step 1: Load the dataset
file_path = '/content/blooms_taxonomy_dataset (1).csv'  # Replace with your dataset file path
df = pd.read_csv(file_path)

# Step 2: Preprocess the dataset
# Clean the 'Questions' column (remove punctuation, convert to lowercase)
df['Questions'] = df['Questions'].str.replace(r'[^\w\s]+', '', regex=True).str.lower()

# Map 'Category' to integers
categories = df['Category'].unique().tolist()  # Get unique categories
df['Label'] = df['Category'].map({cat: i for i, cat in enumerate(categories)})

# Remove rows with missing values
df = df.dropna(subset=['Questions', 'Label'])

# Step 3: Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract texts and labels
train_texts = train_df['Questions'].tolist()
train_labels = train_df['Label'].tolist()
val_texts = val_df['Questions'].tolist()
val_labels = val_df['Label'].tolist()

# Step 4: Tokenize the text data
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# Step 5: Define the model
model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(categories)
)

# Step 6: Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


# Step 7: Train the model
BATCH_SIZE = 16
EPOCHS = 5

train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(len(train_texts))
val_dataset = val_dataset.batch(BATCH_SIZE)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=[callback]
)

# Step 8: Save the model and tokenizer
model.save_pretrained('./blooms_taxonomy_model')
tokenizer.save_pretrained('./blooms_taxonomy_model')

print("Model and tokenizer saved successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
439/439 [==============================] - 4184s 9s/step - loss: 0.8559 - accuracy: 0.6943 - val_loss: 0.6442 - val_accuracy: 0.7691
Epoch 2/5
439/439 [==============================] - 4224s 10s/step - loss: 0.4922 - accuracy: 0.8307 - val_loss: 0.6105 - val_accuracy: 0.7811
Epoch 3/5
439/439 [==============================] - 4161s 9s/step - loss: 0.3486 - accuracy: 0.8742 - val_loss: 0.6468 - val_accuracy: 0.7879
Epoch 4/5
439/439 [==============================] - 4176s 10s/step - loss: 0.2593 - accuracy: 0.9085 - val_loss: 0.6909 - val_accuracy: 0.7925
Epoch 5/5
439/439 [==============================] - 4294s 10s/step - loss: 0.1798 - accuracy: 0.9377 - val_loss: 0.7725 - val_accuracy: 0.8005
Model and tokenizer saved successfully!
